# Hybrid Colocalization Analysis with SpotitPy

This notebook performs hybrid colocalization analysis between mutant Huntingtin (mHTT) aggregates and CCT1 chaperone foci using the SpotitPy library. It combines object-based detection of protein inclusions and puncta with pixel-wise intensity correlation and spatial analysis.

---

**Channels:**
- Channel 1 = mHTT (Red)
- Channel 2 = CCT1 (Green)
- Optional: Channel 3 = Nuclei (Blue)

**Objectives:**
- Segment mHTT aggregates and CCT1 foci
- Compute object-wise overlap and centroid distances
- Stratify results by aggregate size
- Export analysis for downstream biological interpretation

In [ ]:
# Install SpotitPy if not already installed
!pip install spotitpy

In [ ]:
# Load and split multi-channel microscopy image
from spotitpy.utils import load_multichannel_image

image = load_multichannel_image("your_image.tif")  # Replace with your file path
mhtt_img = image[0]  # Red channel: mHTT
cct1_img = image[1]  # Green channel: CCT1

In [ ]:
# Detect CCT1 foci using Laplacian of Gaussian
from spotitpy.spot_detection import detect_spots_log

cct1_spots = detect_spots_log(cct1_img, min_sigma=1, max_sigma=5, threshold=0.03)

In [ ]:
# Detect mHTT aggregates using thresholding
from spotitpy.utils import threshold_objects

mhtt_mask = threshold_objects(mhtt_img, method='otsu', min_size=50, max_size=5000)

In [ ]:
# Compute colocalization overlap and distances
from spotitpy.colocalization import colocalize_spots_objects, distance_to_nearest

# Overlap metrics
overlap_counts, overlap_fraction = colocalize_spots_objects(
    spots=cct1_spots,
    binary_mask=mhtt_mask,
    return_fraction=True
)

# Distance to nearest aggregate
distances = distance_to_nearest(cct1_spots, mhtt_mask)

In [ ]:
# Stratify aggregates by size and associate CCT1 foci
import pandas as pd
from skimage.measure import regionprops, label
import numpy as np
from scipy.spatial import distance_matrix

labeled_mhtt = label(mhtt_mask)
props = regionprops(labeled_mhtt)
agg_sizes = [p.area for p in props]
agg_centroids = [p.centroid for p in props]

df = pd.DataFrame(columns=["cct1_x", "cct1_y", "closest_agg_id", "distance", "agg_area"])

for i, (y, x) in enumerate(cct1_spots):
    dists = distance_matrix([[y, x]], agg_centroids)[0]
    closest = np.argmin(dists)
    df.loc[i] = [x, y, closest, dists[closest], agg_sizes[closest]]

In [ ]:
# Visualization
import matplotlib.pyplot as plt
from spotitpy.utils import overlay_detections

fig, ax = plt.subplots(figsize=(10, 10))
overlay = overlay_detections(mhtt_img, cct1_spots, mhtt_mask)
ax.imshow(overlay)
ax.set_title("CCT1 foci (green) and mHTT aggregates (red outline)")
plt.axis("off")
plt.show()

In [ ]:
# Export results for downstream analysis
df.to_csv("colocalization_per_foci.csv", index=False)